# Step 0: Load Libraries

In [25]:
import pandas as pd
import json
import requests
import csv

import nltk
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

import pickle as pk

import os
from pathlib import Path


folderpath = Path(os.getcwd()).absolute()
data_folderpath = folderpath.parent.joinpath("Data")
model_folderpath = folderpath.parent.joinpath("SentimentModels").joinpath("Inference")
wordnet_folderpath = folderpath.parent.joinpath("WordNet")

print(folderpath)
print(data_folderpath)
print(model_folderpath)
print(wordnet_folderpath)

C:\Users\onnwe\Desktop\ISS_PLP_Project\Sentiment Analysis with Aspect
C:\Users\onnwe\Desktop\ISS_PLP_Project\Data
C:\Users\onnwe\Desktop\ISS_PLP_Project\SentimentModels\Inference
C:\Users\onnwe\Desktop\ISS_PLP_Project\WordNet


# Step 1: Reading Data

In [2]:
filename = "meta_AMAZON_FASHION.json"

filepath = data_folderpath.joinpath(filename)
meta = pd.read_json(filepath, lines=True)

In [3]:
meta

,title,brand,feature,rank,date,asin,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,Slime Time Fall Fest [With CDROM and Collector...,Group Publishing (CO),[Product Dimensions:\n \n8....,"13,052,976inClothing,Shoesamp;Jewelry(",8.70 inches,0764443682,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XCC Qi promise new spider snake preparing men'...,NaN,NaN,"11,654,581inClothing,Shoesamp;Jewelry(",5 star,1291691480,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Magical Things I Really Do Do Too!,Christopher Manos,[Package Dimensions:\n \n8....,"19,308,073inClothing,ShoesJewelry(",5 star,1940280001,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[For the professional or amateur magician. Ro...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Ashes to Ashes, Oranges to Oranges",Flickerlamp Publishing,[Package Dimensions:\n \n8....,"19,734,184inClothing,ShoesJewelry(",5 star,1940735033,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aether & Empire #1 - 2016 First Printing Comic...,NaN,[Package Dimensions:\n \n10...,"10,558,646inClothing,Shoesamp;Jewelry(",5 star,1940967805,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$4.50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186632,JT Women's Elegant Off Shoulder Chiffon Maxi L...,JT,NaN,"9,835,890inClothing,ShoesJewelry(",5 star,B01HJGXL4O,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186633,Microcosm Retro Vintage Black Crochet Lace One...,Microcosm,[Package Dimensions:\n \n7....,"11,390,771inClothing,ShoesJewelry(",5 star5 star (0%),B01HJHF97K,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
186634,Lookatool Classic Plain Vintage Army Military ...,Lookatool,"[Cotton+Polyester, Imported, Item type:Basebal...","972,275inClothing,ShoesJewelry(",5 star,B01HJGJ9LS,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,$8.53,"[B00XLECZMS, B0018MQAOY, B00N833I4Q, B074DQSPP...","[B07BHQ1FXL, B00XLECZMS, B07CJWM5WY, B07CS97C1...","class=""a-normal a-align-center a-spacing-smal...",NaN,NaN,NaN
186635,Edith Windsor Women's Deep V-neck Beaded Sequi...,Edith Windsor,[Product Dimensions:\n \n9....,"1,964,585inClothing,ShoesJewelry(",5 star,B01HJHTH5U,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN


In [4]:
chunksize = 10000

filename = 'AMAZON_FASHION.json'
filepath = data_folderpath.joinpath(filename)

iter = pd.read_json(filepath, chunksize = chunksize, lines = True)

In [5]:
isFirstChunk = True
count = 0

for chunk in iter:
    #print(count)
    if isFirstChunk:
        chunk.to_json("Sample_Review", orient='records', lines=True)
        review_with_meta = pd.merge(chunk, meta, how="left", on="asin")
        review_with_meta["reviewText"] = review_with_meta["reviewText"].replace('\"',' ', regex=True).str.encode('ascii', 'ignore').str.decode('ascii')
        isFirstChunk = False
    else:
        chunk.to_json("Sample_Review", orient='records', lines=True)
        additional_review_with_meta = pd.merge(chunk, meta, how="left", on="asin")
        additional_review_with_meta["reviewText"] = additional_review_with_meta["reviewText"].replace('\"',' ', regex=True).str.encode('ascii', 'ignore').str.decode('ascii')
        review_with_meta = review_with_meta.append(additional_review_with_meta)
    count += 1

In [6]:
review_with_meta

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,5,True,"02 21, 2017",A1ZSB2Q144UTEY,B01HJHTH5U,Amazon Customer,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,1487635200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3632,5,True,"11 25, 2016",A2CCDV0J5VB6F2,B01HJHTH5U,Amazon Customer,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,1480032000,2.0,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3633,3,True,"11 10, 2016",A3O90PACS7B61K,B01HJHTH5U,Fabfifty,Too big in the chest area!,Three Stars,1478736000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B077ZLGMJ3],NaN,NaN,NaN,NaN
3634,3,True,"11 10, 2016",A2HO94I89U3LNH,B01HJHF97K,Mgomez,"Too clear in the back, needs lining",Three Stars,1478736000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 2: Loading Sentiment Analysis Model (Self-trained)

In [10]:
filename = 'vectorizer.pk'
filepath = model_folderpath.joinpath(filename)
vectorizer = pk.load(open(filepath, 'rb'))

filename = 'logreg_model.sav'
filepath = model_folderpath.joinpath(filename)
loaded_model = pk.load(open(filepath, 'rb'))

In [11]:
def logreg_model(TEST_REVIEW, negation_tag):
    test_vectors = vectorizer.transform([TEST_REVIEW])
    predME = loaded_model.predict(test_vectors)
    pred= list(predME)
    if pred==[1.0]:
        if negation_tag:
            prediction = "Negative"
        else:
            prediction = "Positive"
    else:
        if negation_tag:
            prediction = "Positive"
        else:
            prediction = "Negative"
    return prediction

In [12]:
print(logreg_model("Good", False))

Positive


In [249]:
#Trained Spacy Model
def spacy_model(TEST_REVIEW, negation_tag):

    url_start = 'https://danieltanhx.pythonanywhere.com/?input='
    url = url_start+str(TEST_REVIEW)
    
    r = requests.get(url)
    pred = json.loads(r.text)["Predicted sentiment"]
    
    if negation_tag:
        if pred == 'Positive':
            prediction = 'Negative'
        elif pred == 'Negative':
            prediction = 'Positive'
    else:
        prediction = pred
    
    return pred

In [250]:
print(spacy_model("Good", False))

Positive


# Step 3: Determine Nouns and Adjective Keywords For Different Aspects

In [9]:
# mystopwords = stopwords.words("english")
# WNlemma = nltk.WordNetLemmatizer()

# def pre_process(text):
#     text_pos = nltk.pos_tag(nltk.word_tokenize(str(text)))
#     tokens   = [ t for t in lemmaNVAR(text_pos) ]
#     tokens   = [ t for t in tokens if t not in mystopwords]
#     tokens   = [ t for t in tokens if len(t) >= 3 ]
#     return(tokens)

# def lemmaNVAR(wpos):
#     lemmas = []
#     for w, pos in wpos:
#         if pos == 'NNS':
#             lemmas.append(WNlemma.lemmatize(w.lower(), pos = pos[0].lower()))
#     return lemmas

# reviewText = review_with_meta["reviewText"].apply(pre_process)

In [10]:
# my_stopwords =  stopwords.words('english') + ['#']

# def preprocess_noun(doc):
#     doc = nlp(str(doc))
#     noun_list = [token for token in doc if token.pos_== "NOUN" and not token.text.isnumeric()]
#     noun_list = [noun.lemma_ for noun in noun_list if noun.text not in my_stopwords]

#     return noun_list

# def preprocess_adjective(doc):
#     doc = nlp(str(doc))
#     adjective_list = [token for token in doc if token.pos_ == "ADJ" and not token.text.isnumeric()]
#     adjective_list = [adjective.lemma_ for adjective in adjective_list if adjective.text not in my_stopwords]
    
#     return adjective_list

In [11]:
# noun_list = preprocess_noun(review_with_meta["reviewText"].iloc[1])
# adjective_list = preprocess_adjective(review_with_meta["reviewText"].iloc[1])

# print(noun_list)
# print(adjective_list)

['review', 'opening', 'hook', 'earring', 'end', 'price']
['small', 'expensive', 'high']


In [12]:
# import time
# start_time = time.time()
# noun_list = review_with_meta["reviewText"].apply(preprocess_noun)
# end_time = time.time()
# print(f"Time taken {end_time-start_time}")

Time taken 5065.516253948212


In [13]:
# import time
# start_time = time.time()
# adjective_list = review_with_meta["reviewText"].apply(preprocess_adjective)
# end_time = time.time()
# print(f"Time taken {end_time-start_time}")

Time taken 4935.9662482738495


In [14]:
# noun_toks = [toks for tokens in noun_list for toks in tokens]
# fdist = FreqDist(noun_toks)
# fdist.most_common(100)

[('size', 147658),
 ('quality', 84732),
 ('color', 69443),
 ('dress', 68854),
 ('shirt', 62614),
 ('price', 59233),
 ('material', 58669),
 ('product', 57751),
 ('time', 53015),
 ('picture', 40356),
 ('bit', 38459),
 ('top', 37026),
 ('fit', 34854),
 ('one', 33885),
 ('day', 32619),
 ('review', 32392),
 ('ring', 29979),
 ('way', 29466),
 ('shoe', 29085),
 ('year', 28858),
 ('daughter', 28344),
 ('fabric', 28222),
 ('hat', 28190),
 ('pair', 26899),
 ('bag', 26738),
 ('lot', 25455),
 ('item', 25403),
 ('gift', 23940),
 ('thing', 22321),
 ('month', 21587),
 ('medium', 21358),
 ('pant', 20744),
 ('belt', 18589),
 ('money', 17995),
 ('pocket', 17696),
 ('compliment', 17355),
 ('bottom', 17296),
 ('side', 17235),
 ('son', 16899),
 ('waist', 16800),
 ('star', 16697),
 ('foot', 16277),
 ('wallet', 16069),
 ('design', 15851),
 ('style', 15704),
 ('length', 15430),
 ('purchase', 15363),
 ('week', 15323),
 ('piece', 15192),
 ('suit', 14916),
 ('strap', 14601),
 ('problem', 13849),
 ('necklace', 13

In [15]:
# adjective_toks = [toks for tokens in adjective_list for toks in tokens]
# fdist = FreqDist(adjective_toks)
# fdist.most_common(100)

[('great', 153509),
 ('small', 131651),
 ('good', 112526),
 ('nice', 96258),
 ('little', 71422),
 ('large', 69101),
 ('perfect', 67577),
 ('cute', 63769),
 ('big', 61944),
 ('comfortable', 50356),
 ('beautiful', 40741),
 ('cheap', 34467),
 ('fit', 32519),
 ('soft', 29777),
 ('long', 28413),
 ('short', 26075),
 ('tight', 25992),
 ('happy', 25985),
 ('old', 25805),
 ('thin', 23655),
 ('different', 19120),
 ('well', 18450),
 ('first', 16898),
 ('many', 16612),
 ('warm', 16389),
 ('easy', 16097),
 ('sure', 15922),
 ('high', 15475),
 ('worth', 15422),
 ('bad', 15386),
 ('pretty', 15037),
 ('black', 14962),
 ('light', 14061),
 ('right', 13950),
 ('loose', 13863),
 ('true', 13239),
 ('disappointed', 13079),
 ('excellent', 12646),
 ('thick', 12443),
 ('new', 11834),
 ('fine', 11758),
 ('much', 11705),
 ('awesome', 11179),
 ('white', 11047),
 ('able', 10926),
 ('comfy', 10532),
 ('extra', 10447),
 ('heavy', 10443),
 ('amazing', 10356),
 ('hard', 10091),
 ('pleased', 10024),
 ('cool', 9154),
 ('

# Step 4: Model

In [243]:
topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']

In [87]:
# Aspect Types from common Amazon data
noun_keywords = { 'size'          : ['size', 'fit', 'length'],
                  'comfort'       : [],
                  'appearance'    : ['colour', 'picture', 'design', 'style', 'photo'],
                  'quality'       : ['quality', 'material', 'fabric', 'leather'],
                  'price'         : ['price', 'money'],
                  'delivery'      : ['time', 'day', 'seller', 'shipping', 'order']}
                     
for topic in topic_list:
    filename = topic+'.csv'
    filepath = wordnet_folderpath.joinpath(topic).joinpath(filename)
    
    with open(filepath, newline='\n') as f:
        reader = csv.reader(f)
        data = list(reader)
    nouns = noun_keywords[topic] + [pair[0] for pair in data if pair[1] == 'noun']
    noun_keywords[topic] = list(dict.fromkeys(nouns))
    print(noun_keywords[topic])

['size', 'fit', 'length', 'medium', 'bulkiness', 'enormousness', 'vastness', 'hang', '8vo', 'tininess', 'enormity', '4to', 'minuteness', 'narrow gauge', 'airiness', 'octavo', 'wingspan', 'voluminousness', 'slightness', 'diminutiveness', 'wingspread', 'gigantism', "arm's length", 'number', 'delicacy', 'stuntedness', 'capaciousness', 'giantism', 'spaciousness', 'circumference', 'skip distance', 'seating capacity', 'ampleness', 'littleness', 'greatness', 'grandness', 'spread', 'petite', 'wideness', 'stout', 'extra large', 'gauge', 'regular', 'immenseness', 'dovetail', 'perimeter', 'dwarfishness', 'bigness', 'broad gauge', 'quarto', 'runtiness', 'girth', 'tall', 'small', 'sizeableness', 'smallness', 'petiteness', 'leg', 'large', 'roominess', 'fitting', 'puniness', 'immensity', 'grain', 'commodiousness', 'standard gauge', 'joint', 'voluminosity', 'light time', 'fullness', 'outsize', 'weeness', 'eightvo', 'distance', 'largeness', 'massiveness', 'fitter', 'altitude']
['solace', 'alleviation',

In [88]:
# Aspect Types from common Amazon data
adjective_keywords = { 'size'          : ['small', 'large', 'little', 'big', 'fit', 'long', 'short', 'tight', 'loose', 'medium', 'tiny', 'huge'],
                       'comfort'       : ['comfortable', 'uncomfortable', 'soft', 'lightweight','comfy'],
                       'appearance'    : ['beautiful', 'stylish', 'flattering', 'gorgeoous', 'lovely', 'sexy', 'adorable', 'cool'],
                       'quality'       : ['durable', 'sturdy', 'heavy', 'thick', 'new', 'old', 'hard'],
                       'price'         : ['cheap', 'expensive', 'honest', 'worth'],
                       'delivery'      : ['fast', 'quick']}

for topic in topic_list:
    filename = topic+'.csv'
    filepath = wordnet_folderpath.joinpath(topic).joinpath(filename)
    
    with open(filepath, newline='\n') as f:
        reader = csv.reader(f)
        data = list(reader)
    adjectives = adjective_keywords[topic] + [pair[0] for pair in data if pair[1] == 'adj']
    adjective_keywords[topic] = list(dict.fromkeys(adjectives))
    print(adjective_keywords[topic])

['small', 'large', 'little', 'big', 'fit', 'long', 'short', 'tight', 'loose', 'medium', 'tiny', 'huge', 'runty', 'voluminous', 'diminutive', 'average', 'commodious', 'medial', 'stunted', 'puny', 'sizeable', 'wee', 'immense', 'massive', 'outsize', 'great', 'dwarfish', 'ample', 'wide', 'vast', 'airy', 'altitudinal', 'intermediate', 'capacious', 'roomy', 'enormous', 'minute', 'slight', 'grand', 'full', 'petite', 'bulky', 'stout', 'spacious']
['comfortable', 'uncomfortable', 'soft', 'lightweight', 'comfy', 'cozy', 'cosy', 'snug', 'convenient']
['beautiful', 'stylish', 'flattering', 'gorgeoous', 'lovely', 'sexy', 'adorable', 'cool', 'unsightly', 'showy', 'fair', 'mellow', 'glorious', 'grey', 'homely', 'sapphire', 'rosy', 'vinaceous', 'out-of-date', 'nice', 'florid', 'outdated', 'dark', 'cerise', 'greenish', 'ultramarine', 'crimson', 'stripy', 'splashy', 'lurid', 'wan', 'gray', 'garish', 'iron-grey', 'iron-gray', 'azure', 'rich', 'straw', 'chestnut', 'blotchy', 'sanguineous', 'superficial', 

In [89]:
def find_noun_head(token):
    #print(f"Finding Token: {token}")
    #print(f"Token Head: {token.head}")
    if token.head.pos_ == "NOUN" or token.head.pos_ == "NOUN" or token.head.dep_ == "ROOT":
        #print("Found")
        return token.head
    else:
        #print("Continue Search")
        return find_noun_head(token.head)
    
def find_negation_tag(adjective):
    #print(f"Finding Negatation for Adjective: {adjective}")
    #print(f"Token Head: {adjective.head}")
    if adjective.head.dep_ == "neg":
        return True
    elif adjective.head.pos_ == "AUX":
        for child in adjective.head.children:
            if child.dep_ =='neg':
                return True
    return False

def find_span_start(sent, token):
    #for tok in sent:
    #    print(f"{tok.i}: {tok} - {tok.pos_}")
    #print(f"Start {token.i}: {token} - {token.pos_}")  
    if token.i == sent.start or sent[token.i-1].pos_ == "PUNCT":
        #print(token.i)
        return token.i
    else:
        #print(token.i-1)
        return find_span_start(sent, sent[token.i-1])

def find_span_end(sent, token):
    #for tok in sent:
    #    print(f"{tok.i}: {tok} - {tok.pos_}")
    #print(f"End {token.i}: {token} - {token.pos_}")        
    if token.i+1 == sent.end or sent[token.i+1].pos_ == "PUNCT":
        #print(token.i+1)
        return token.i+1
    else:
        #print(token.i+1)
        return find_span_end(sent, sent[token.i+1])
    
def match_topics(noun_token, adjective_token, topic_list, noun_keywords, adjective_keywords):

    for topic in topic_list:
        if noun_token.text in noun_keywords[topic]:
            return topic
        elif adjective_token.text in adjective_keywords[topic]:
            return topic

    return None

In [205]:
def adsa_model(review, SENTI_MODEL = 'logreg', DEBUG = False):

    #Model Setting
    SENTI_MODEL = SENTI_MODEL
    DEBUG = DEBUG

    #Model    
    doc = nlp(review)

    topic_prediction = {'size'          : None,
                        'comfort'       : None,
                        'appearance'    : None,
                        'quality'       : None,
                        'price'         : None, 
                        'delivery'      : None}

    sent_count = 0
    for sent in doc.sents:
        sent_count += 1

        descriptor_pair = []
        adjectives = [tok for tok in sent if tok.pos_ == "ADJ"]
        pronouns = [tok for tok in sent if tok.pos_ == "PRON"]
        nouns = [tok for tok in sent if tok.pos_ == "NOUN"]
        negations = [tok for tok in sent if tok.dep_ == "neg"]

        for adjective in adjectives:
            isFound = False
            topic = None
            
            try:
                descriptor = ""
                for child in adjective.children:
                    if child.pos_ == "ADV":
                        descriptor += child.text + " "
                descriptor += adjective.text

                negation_tag = find_negation_tag(adjective)


                #Direct (i.e. The dress has a beautiful colour)
                noun = find_noun_head(adjective)
                if noun.pos_ == "NOUN" or noun.pos_ == "PRON":
                    trace = f"Direct Reference Detected for Adjective: {adjective}"
                    for chunk in sent.noun_chunks:
                        if chunk.root == noun and adjective not in chunk:
                            isFound = True
                            noun_subj = chunk
                        else:
                            isFound = True
                            noun_subj = noun             

                #Passive Voice (i.e. Colour of the dress was beautiful)
                elif adjective.head.pos_ == "AUX" or adjective.head.pos_ == "VERB":
                    trace = f"Indirect Reference Detected for Adjective: {adjective}"
                    for child in adjective.head.children:
                        if child.pos_ == "NOUN" or child.pos_ == "PRON":
                            noun = child                      
                            for chunk in sent.noun_chunks:
                                if chunk.root == child and adjective not in chunk:
                                    isFound = True
                                    noun_subj = chunk
                                else:
                                    isFound = True
                                    noun_subj = noun
                                    
                #Guessing when improper grammer is used (i.e. Colour of the dress was beautiful vs also dress colour beautiful)
                else:
                    trace = f"Guessing for Adjective: {adjective}\n"
                    start = find_span_start(sent, adjective)
                    end = find_span_end(sent, adjective)
                    extract = sent[start:end]
                    trace += f"Extract: {extract}"

                    for token in extract:
                        if token.pos_ == "NOUN" and token.dep_ == "nsubj":
                            isFound = True
                            noun_subj = token
                        if token.dep_ == "neg":
                            negation_tag = True
            except:
                isFound = False

            if isFound:
                topic = match_topics(noun, adjective, topic_list, noun_keywords, adjective_keywords)
            else:
                noun_subj = None
                topic = match_topics(nlp(""), adjective, topic_list, noun_keywords, adjective_keywords)

            if topic != None:
                if SENTI_MODEL == 'logreg':
                    prediction = logreg_model(str(descriptor) + " " + str(noun_subj), negation_tag)
                elif SENTI_MODEL == 'spacy':
                    prediction = spacy_model(str(descriptor) + " " + str(noun_subj), negation_tag)

                descriptor_pair.append((descriptor, noun_subj, negation_tag, topic, prediction))
                topic_prediction[topic] = prediction

            if DEBUG:
                print(f"Sentence {sent_count}: {sent}")
                print(f"Function trace: {trace}")
                displacy.render(sent, style="dep")
                print(f"Adjectives: {adjectives}")
                print(f"Nouns: {nouns}")
                print(f"Pronouns: {pronouns}")
                print(f"Negations: {negations}")
                print(f"Descriptor_pair: {descriptor_pair}")
                print("")

    print(f"Review: {review}")
    print(f"Topic_prediction: {topic_prediction}")
    print("---------------------------------------------------------------")
    print("")
    
    return topic_prediction

# Step 5: Apply Model to Evaluation Dataset

In [215]:
filename = 'amazon_absa_ner.jsonl'
filepath = data_folderpath.joinpath(filename)
test_data = pd.read_json(filepath, lines=True)

In [216]:
def process_evaluation(spans):
    
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']    
    topic_prediction = {'size'          : None,
                        'comfort'       : None,
                        'appearance'    : None,
                        'quality'       : None,
                        'price'         : None, 
                        'delivery'      : None}
    
    for span in spans:
        topic_prediction[span['label'].lower()] = "Found"
        
    return topic_prediction    

In [217]:
test_data['topics_target'] = test_data['spans'].apply(process_evaluation)

In [218]:
test_data['topics_predicted'] = test_data['text'].apply(adsa_model)

Review: nut Malo es very cheap
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Negative', 'delivery': None}
---------------------------------------------------------------

Review: It seems the sizing of this is way off.  I read the reviews and ordered a size larger than I'd normally wear and it was still WAY TOO SMALL!
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: they are too small they did not even fit on my ear i would not recommend this product they are so tiny its a joke
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I thought I got the right size but apparently not. WAY too small. Order up a size (I'm a cross dresser, and this looked cut

Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Overall this is a very cute shirt, I followed the recommend sizing from other reviewers and the shirt was a bit too big. It is not a slimming shirt, it would be great if you are pregnant as it has a lot of extra room. I tried using a belt and it didn't help at all. It doesn't hang like it shows in the picture. I do love the color and the chest and sleeves did fit well. It also only hangs to about your waste so even making it work with leggings wasn't happening for me. I would either try to size to your current size or even size down. It hangs extremely loosely past your chest. It could be that I am picky about fit. I would recommend this product to anyone who doesn't mind a loose fit, the color is beautiful and it may fit better with a smaller size which is why I am giving it four stars.

I

Review: Nice
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: The picture looks more expensive than the real thing. It changed color after the second wear!!  Waste of money!
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: This wallet is larger than I expected, but still is a great wallet and I love it. It is also a little thicker than i expected. It looks a little thinner in the picture. But 10/10 I would recommend to a friend.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: This hoodie is amazing! It arrived when it was supposed to, and the shade 

Review: Loved it! Perfect size and looks great! Holds everything without being to bulky!
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: The sweatshirt is super cute. I wear a S in tops normally. I ordered a M for the sweatshirt, so I can layer. The M fits like a S and I won't be able to wear much underneath it. Still, very happy with the sweatshirt.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Darling tshirt
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: These run a liittle small and are too tight. Fabric was fine.
Topic_prediction: {'size': 

Review: Not what I expected
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: My son is a large 4 months. This is big on him, as was expected since it is a 3-6 month size. Kept him warm. Easy to get on and off for swim and to change his swim diaper.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: LOVE IT
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Beautiful stole!  Just what I expected.  Glad I ordered it.  Good quality and just the right length.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': 'Positive', '

...

Review: I'm 5'1, waist 26, B cup and weight 115. I got a size large and while the length is perfect, it is a kinda wide so both the hip straps and the waist fit a bit loose. I'm not sure whether the medium would've fit better or it would've been too tight length wise. Either way, I love it, it looks so pretty, the strap colors are very vibrant, and it's good quality.
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': 'Positive', 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: good
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Very pleased.. I ordered 3 dresses and recieved them in just a few weeks and they were absolutely beautiful and fit good.  Would definitely order from them again ...they will look beautiful on my bridesmaids for my weddin

Review: These work well in my Brooks walkers, and my dress shoes. I found them to be average in my hiking boots, but perform well in the other two shoes. These work well for Plantar Fascitis, and fallen arches.
Topic_prediction: {'size': 'Negative', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: It was smaller than expected
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Smaller than last pair
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Very pretty styling with modest coverage.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': 'Positive', 'd

Review: They are beautiful just the right color.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: Fell apart after 1 use.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: very nice christmas gift sweet wolf that is great fits nice  my son loves wolf of all kind.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: very nice...quick shipment
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Positive', 'quality': None, 'price': None, 'delivery': 'Positive'}
---------------------------------------------------------------

Review: Lovely

Review: HALLOWEEN 2016: This is a pretty good Halloween costume for the price. I read the other reviews and was worried about the costume sizing and the size of the coon skin hat. Luckily as of September 2016 the manufacturer has fixed these issues.

 Costume Fabric 

The fabric is a Polyester faux suede. For lack of a better way to describe it, the material is like a thin shammy-type microfiber towel. You know, the ones you drape around your neck that everyone has. It reminds me of that material but it is only suede-like on one side. It is too thin for warmth but too thick to truly breathe properly. We live in Houston and my son started feeling too hot in it during 80 degree weather but it is also too lightweight to provide warmth if it is cold. Plan on layering something under this if you have to trick or treat in anything colder than 65F. The fringe is even thinner and lighter weight but it does look cute. It is prone to wrinkling and crumpling a bit more but nothing a bit of steam 

Review: Perfect for prom. Everyone loved my dress and it fit perfectly. Looks better than the picture. I would definitely recommend this to anyone
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Review: I had to sow one of the ties back on but other than that it is perfect.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------



In [220]:
def accuracy(series):
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']    
    
    correct = 0
    for topic in topic_list:
        if series['topics_target'][topic] != None and series['topics_predicted'][topic] != None:
            correct += 1
        elif series['topics_target'][topic] == None and series['topics_predicted'][topic] == None:
            correct += 1
    
    accuracy = correct / len(topic_list)
    
    return accuracy

In [221]:
test_data['accuracy'] = test_data[['topics_target', 'topics_predicted']].apply(accuracy, axis=1)

In [226]:
def to_list(dict):
    topic_list = ['size', 'comfort', 'appearance', 'quality', 'price', 'delivery']
    
    list = []
    
    for topic in topic_list:
        if dict[topic] != None:
            list.append(1)
        else:
            list.append(0)
    
    return list

In [227]:
test_data['target_list'] = test_data['topics_target'].apply(to_list)
test_data['predicted_list'] = test_data['topics_predicted'].apply(to_list)

In [234]:
test_data

,text,_input_hash,_task_hash,tokens,spans,_is_binary,_session_id,_view_id,answer,_timestamp,topics_target,topics_predicted,accuracy,target_list,predicted_list
0,nut Malo es very cheap,253581581,236644319,"[{'text': 'nut', 'start': 0, 'end': 3, 'id': 0...","[{'start': 12, 'end': 22, 'token_start': 3, 't...",False,NaN,ner_manual,accept,1636195452,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",1.000000,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0]"
1,It seems the sizing of this is way off. I rea...,-370550970,1479148072,"[{'text': 'It', 'start': 0, 'end': 2, 'id': 0,...","[{'start': 13, 'end': 38, 'token_start': 3, 't...",False,NaN,ner_manual,accept,1636195489,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
2,they are too small they did not even fit on my...,-577070996,415105093,"[{'text': 'they', 'start': 0, 'end': 4, 'id': ...","[{'start': 9, 'end': 18, 'token_start': 2, 'to...",False,NaN,ner_manual,accept,1636195524,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': None, 'appeara...",1.000000,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]"
3,I thought I got the right size but apparently ...,1772326786,-1590407243,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...","[{'start': 51, 'end': 64, 'token_start': 11, '...",False,NaN,ner_manual,accept,1636195549,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': None, 'appeara...",1.000000,"[1, 0, 1, 0, 0, 0]","[1, 0, 1, 0, 0, 0]"
4,I love this wallet,137545609,1515966090,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...",[],False,NaN,ner_manual,accept,1636195554,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",1.000000,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,The dress had a very strong smell of factory. ...,-533040091,-1844490635,"[{'text': 'The', 'start': 0, 'end': 3, 'id': 0...","[{'start': 16, 'end': 44, 'token_start': 4, 't...",False,NaN,ner_manual,accept,1636257683,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 1, 1, 0, 0]","[0, 0, 1, 0, 0, 0]"
456,Nice for the money. This is for Xmas.,-67636620,1347205755,"[{'text': 'Nice', 'start': 0, 'end': 4, 'id': ...","[{'start': 0, 'end': 18, 'token_start': 0, 'to...",False,NaN,ner_manual,accept,1636257687,"{'size': None, 'comfort': None, 'appearance': ...","{'size': None, 'comfort': None, 'appearance': ...",0.833333,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 0, 0]"
457,I am so THRILLED about this dress! The fabric...,1897674455,209258134,"[{'text': 'I', 'start': 0, 'end': 1, 'id': 0, ...","[{'start': 40, 'end': 72, 'token_start': 10, '...",False,NaN,ner_manual,accept,1636257706,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': 'Negative', 'comfort': None, 'appeara...",0.833333,"[1, 0, 1, 1, 0, 0]","[1, 0, 1, 1, 1, 0]"
458,Perfect for prom. Everyone loved my dress and ...,-1550624492,181493175,"[{'text': 'Perfect', 'start': 0, 'end': 7, 'id...","[{'start': 46, 'end': 62, 'text': 'it fit perf...",False,NaN,ner_manual,accept,1636257712,"{'size': 'Found', 'comfort': None, 'appearance...","{'size': None, 'comfort': None, 'appearance': ...",0.666667,"[1, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"


In [240]:
target_list = []
predicted_list = []

for index in range(len(test_data)):
    target_list.append(test_data.iloc[index]['target_list'])
    predicted_list.append(test_data.iloc[index]['predicted_list'])

In [248]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

print(multilabel_confusion_matrix(target_list, predicted_list))
print(classification_report(target_list, predicted_list))

[[[221  48]
  [ 61 130]]

 [[385   3]
  [ 27  45]]

 [[265  25]
  [ 72  98]]

 [[315  25]
  [ 71  49]]

 [[409  17]
  [ 13  21]]

 [[417   9]
  [ 20  14]]]
              precision    recall  f1-score   support

           0       0.73      0.68      0.70       191
           1       0.94      0.62      0.75        72
           2       0.80      0.58      0.67       170
           3       0.66      0.41      0.51       120
           4       0.55      0.62      0.58        34
           5       0.61      0.41      0.49        34

   micro avg       0.74      0.57      0.65       621
   macro avg       0.71      0.55      0.62       621
weighted avg       0.74      0.57      0.64       621
 samples avg       0.49      0.44      0.44       621



c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\onnwe\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [222]:
index = 1

print(test_data.iloc[index]['text'])
print(test_data.iloc[index]['topics_target'])
print(test_data.iloc[index]['topics_predicted'])
print(test_data.iloc[index]['accuracy'])

It seems the sizing of this is way off.  I read the reviews and ordered a size larger than I'd normally wear and it was still WAY TOO SMALL!
{'size': 'Found', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
{'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
0.8333333333333334


In [223]:
import numpy as np

np.average(test_data['accuracy'])

0.8583333333333334

# Step 6: Apply Model to Selected Brands

In [201]:
brand_name = "adidas"

print(meta[meta["brand"]==brand_name])

                                                   title   brand  \
1512     adidas Women's ClimaCool Superlite No Show Sock  adidas   
5812    adidas Women's Response Short Sleeve Stretch Tee  adidas   
6126       adidas Women's Essentials 3-Stripe Knit Short  adidas   
6132              adidas Women's Essentials 3-Stripe Tee  adidas   
6384        ADIDAS Calissage black/metallic silver/white  adidas   
8934                                    adidas ZX 800 DB  adidas   
9882                    Adidas Attaak II Wrestling Shoes  adidas   
10392            adidas Women's Techfit Short-Sleeve Top  adidas   
11609  adidas Women's Ultimate Workout Short-Sleeve V...  adidas   
13699               Adidas Adi Firebird Tracktop #X41202  adidas   
57804                       adidas Boys' Triple Up Short  adidas   
69443  adidas Outdoor Kids Baby Girl's Daroga Leather...  adidas   
70852  adidas Women's Climacool Aeroknit Long Sleeve ...  adidas   
83933  adidas Manchester United FC Official 2015

In [202]:
reduced_review_with_meta = review_with_meta[review_with_meta["brand"]==brand_name]

In [203]:
reduced_review_with_meta

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,imageURL,imageURLHighRes,description,price,also_view,also_buy,fit,details,similar_item,tech1
3411,5,True,"05 17, 2016",AOU7T6BK2IQ1J,B00N9KLTSK,Byoung K.,Good,Five Stars,1463443200,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3412,5,True,"02 9, 2016",A2V6HAVTGVWA8S,B00N9KLTSK,Kara M Lauve,Perfect fit,Five Stars,1454976000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3413,5,True,"02 4, 2016",A3GD1CS79C1JBN,B00N9KLTSK,sugarnay,works great. My son likes them.,Five Stars,1454544000,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3414,5,True,"01 24, 2016",A2GCYCPA9VOEFR,B00N9KLTSK,Dustin P. Ward,they run a little large. No worries just roll ...,Sweet looking and very comfortable.,1453593600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3415,5,True,"12 27, 2015",A2ISCKJ7OLT1QO,B00N9KLTSK,Amazon Customer,My son loves these!,Five Stars,1451174400,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3416,5,True,"11 30, 2015",AU1BSTL6EUKP,B00N9KLTSK,Jenna,My son loves basketball shorts. He would wear ...,Great shorts,1448841600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3417,5,True,"10 31, 2015",A38C8XNIETJLEU,B00N9KLTSK,Pen Name,"My son love it, the fabric also nice and i'll ...",littile big,1446249600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Children will be excited to get active in fas...,$9.95 - $24.99,"[B07D6MJRRL, B015SSQCO6, B01HO66CPE, B00CHNS6Y...","[B07D6MJRRL, B00U9TWIJS, B07CS9VTPB, B00CHNS6Y...","class=""a-normal a-align-center a-spacing-smal...","\n <div class=""content"">\n\n\n\n\n\n\n<ul...",NaN,NaN
3251,5,True,"09 21, 2015",A24YEGDNPYJGO,B00VJX7AC8,yamilia olivera rivera,Thank you,Five Stars,1442793600,NaN,...,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,[Manchester United Home Jersey 2015 / 2016],$43.56

In [206]:
for index in range(len(reduced_review_with_meta)):
    (brand, product, review, rating) = (str(reduced_review_with_meta.iloc[index]["brand"]), 
                                        str(reduced_review_with_meta.iloc[index]["title"]), 
                                        str(reduced_review_with_meta.iloc[index]["reviewText"]),
                                        str(reduced_review_with_meta.iloc[index]["overall"]))

    print(f"Index: {index}")
    print(f"Brand: {brand}")
    print(f"Product: {product}")
    print(f"Review Text: {review}")
    print(f"Product Rating: {rating}")
    adsa_model(review)

Index: 0
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: Good
Product Rating: 5
Review: Good
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 1
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: Perfect fit
Product Rating: 5
Review: Perfect fit
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 2
Brand: adidas
Product: adidas Boys' Triple Up Short
Review Text: works great. My son likes them.
Product Rating: 5
Review: works great. My son likes them.
Topic_prediction: {'size': 'Positive', 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 3
Brand: adidas
Product: adi

Review: Great fabric, soft, adjust to your shape. If you don't like it to tight buy next size.
I will buy more.
Topic_prediction: {'size': 'Positive', 'comfort': 'Positive', 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 17
Brand: adidas
Product: adidas Women's Ultimate Workout Short-Sleeve V-Neck Top
Review Text: el producto es excelente lo recibi en buenas condiciones y realmente fue lo que me ofrecieron. lo recomiendo ampliamente gracias.
Product Rating: 5
Review: el producto es excelente lo recibi en buenas condiciones y realmente fue lo que me ofrecieron. lo recomiendo ampliamente gracias.
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 18
Brand: adidas
Product: adidas Women's Essentials 3-Stripe Tee
Review Text: la ropa es de la talla que 

Review: On the picture appeared a white shirt with red stripe and I received a blue with dark blue stripe one. I wanted the white.
I am not happy with this mistake
Topic_prediction: {'size': None, 'comfort': None, 'appearance': 'Negative', 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 37
Brand: adidas
Product: adidas Women's Climacool Aeroknit Long Sleeve Tee, Black Heather, 2XL
Review Text: Great product
Product Rating: 5
Review: Great product
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}
---------------------------------------------------------------

Index: 38
Brand: adidas
Product: adidas Manchester United FC Official 2015/16 SS Away Jersey - Adult - White -
Review Text: Very good
Product Rating: 4
Review: Very good
Topic_prediction: {'size': None, 'comfort': None, 'appearance': None, 'quality': None, 'price': None, 'delivery': None}


In [224]:
adsa_model("My son loves basketball shorts. He would wear them nonstop if I let him. I purchased these for him, and they are perfect. The fit is accurate, and the quality is excellent. They wash and hold up very well. I have purchased these again because he likes them and says they are very comfortable.")

Review: My son loves basketball shorts. He would wear them nonstop if I let him. I purchased these for him, and they are perfect. The fit is accurate, and the quality is excellent. They wash and hold up very well. I have purchased these again because he likes them and says they are very comfortable.
Topic_prediction: {'size': 'Positive', 'comfort': 'Positive', 'appearance': None, 'quality': 'Positive', 'price': None, 'delivery': None}
---------------------------------------------------------------



{'size': 'Positive',
 'comfort': 'Positive',
 'appearance': None,
 'quality': 'Positive',
 'price': None,
 'delivery': None}